In [1]:
import pandas as pd
import re

In [2]:
sharks = pd.read_csv('./GSAF5.csv', sep = ',', engine='python') #Usar engine='python', porque si no da error de lectura.

In [3]:
display(sharks.head())

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [4]:
print(sharks.columns)

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')


In [5]:
display(sharks.isnull().sum()) 
# Podemos ver que las columnas 'Unnamed: 22' y 'Unnamed: 23' vienen con casi todos los registros vacíos.

Case Number                  0
Date                         0
Year                         0
Type                         0
Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
pdf                          0
href formula                 1
href                         3
Case Number.1                0
Case Number.2                0
original order               0
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

In [6]:
sharks_copy = sharks.copy()

In [7]:
#Elimino columnas 'Unnamed: 22' y 'Unnamed: 23', puesto que vienen sin datos.
sharks_copy.drop(['Unnamed: 22', 'Unnamed: 23'], axis=1, inplace = True)
sharks_copy.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,N,NaN,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,N,NaN,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989


In [8]:
display(sharks[(sharks['Case Number']!= sharks['Case Number.1'])|(sharks['Case Number']!= sharks['Case Number.2'])][['Case Number', 'Case Number.1', 'Case Number.2']])
#Comprobamos que únicamente hay 13 registros en los que la info de las columnas es diferente, 
# y se puede ver que difiere en formato (. , /), que ya sustituiré más adelante si fuera necesario

,Case Number,Case Number.1,Case Number.2
4,2016.09.15,2016.09.16,2016.09.15
33,2016.07.14.4,2016.07.14.R,2016.07.14.4
97,2016.01.24.b,2015.01.24.b,2016.01.24.b
116,2015.12.23,2015.11.07,2015.12.23
121,2015.10.28.a,2015.10.28,2015.10.28.a
169,2015.07-10,2015.07.10,2015.07.10
3296,1967.07.05,1967/07.05,1967.07.05
3569,"1962,08.30.b",1962.08.30.b,"1962,08.30.b"
3654,1961.09.02.R,"1961.09,06.R",1961.09.02.R
4177,1952.08.05,1952.08.04,1952.08.05


In [9]:
#Elimino columnas 'Case Number.1' y 'Case Number.2', puesto que sus valores son los mismos que los de la columna Case Number.
sharks_copy.drop(['Case Number.1', 'Case Number.2'], axis=1, inplace = True)
sharks_copy.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5989


In [10]:
#Elimino columnas 'pdf', puesto que sus valores son los nombres de los archivos, pero no disponemos de ellos.
sharks_copy.drop(['pdf'], axis=1, inplace = True)
sharks_copy.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,href formula,href,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,"The Age, 9/18/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,"The Age, 9/16/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5989


In [11]:
sharks_copy[['Case Number','Investigator or Source']].head()

,Case Number,Investigator or Source
0,2016.09.18.c,"Orlando Sentinel, 9/19/2016"
1,2016.09.18.b,"Orlando Sentinel, 9/19/2016"
2,2016.09.18.a,"Orlando Sentinel, 9/19/2016"
3,2016.09.17,"The Age, 9/18/2016"
4,2016.09.15,"The Age, 9/16/2016"


In [12]:
#Para el análisis que voy a realizar no necesito saber quién ha realizado el estudio ni lo ha publicado. DELETE

In [13]:
sharks_copy.drop(['Investigator or Source'], axis=1, inplace = True)
sharks_copy.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,href formula,href,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5989


In [14]:
display(sharks[(sharks['href formula']!= sharks['href'])][['href formula', 'href']].shape)


(54, 2)

In [15]:
#Comprobamos que las columnas href y href formula difieren, por lo que voy a eliminar la columna href.
sharks_copy.drop(['href'], axis=1, inplace = True)
sharks_copy.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,href formula,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...,5989


In [16]:
sharks["Year"].describe() #Distribución por año del suceso
#Hay datos con año 0, empiezo a ver la información que viene en la columna 'date' de los casos con año 0. (sharks_0)

count    5992.000000
mean     1925.204606
std       286.473712
min         0.000000
25%      1942.000000
50%      1975.000000
75%      2003.000000
max      2016.000000
Name: Year, dtype: float64

In [17]:
sharks_copy[sharks_copy["Year"] == 0].shape
#Tan solo hay 124 registros con Year == 0--> DELETE
cols_to_delete= sharks_copy[(sharks_copy['Year']==0)]
cols_to_delete['Case Number'].head()

5868     0.0214
5869     0.0336
5870     0.0493
5871     0.0725
5872    ND-0153
Name: Case Number, dtype: object

In [18]:
sharks_copy.shape

(5992, 17)

In [19]:
sharks_copy.drop(cols_to_delete.index, axis=0, inplace = True)
sharks_copy.shape

(5868, 17)

In [20]:
#Funciones para sacar el día,mes y año del suceso desde el campo Case Number.

a='2016.09.18.c'
b='2016.09.17'
def year(string):
    dev=string[0:4]
    return int(dev)
def month(string):
    dev=string[5:7]
    return int(dev)
def day(string):
    dev=string[8:10]
    return int(dev)

#def day(string):
 #   dev=string
  #  w= re.findall('\.\w$',string)
   # w = str(w)
    #dev = re.sub(w,'',dev)
    #dev = dev[6:]
    #return dev
#c[:4] --- YEAR
day(a)

18

In [21]:
sharks_copy['Year2']=sharks_copy['Case Number'].apply(year)
sharks_copy['Year2'].value_counts().head()

2015    139
2011    128
2014    125
2013    122
2008    121
Name: Year2, dtype: int64

In [22]:
sharks_copy['Month']=sharks_copy['Case Number'].apply(month)
sharks_copy['Month'].value_counts()

7     634
8     580
9     524
1     500
6     474
12    423
4     410
10    407
3     398
0     398
11    393
5     368
2     359
Name: Month, dtype: int64

In [23]:
sharks_copy.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'href formula', 'original order', 'Year2', 'Month'],
      dtype='object')

In [24]:
display(sharks_copy[(sharks_copy['Year']!= sharks_copy['Year2'])][['Year', 'Year2']])
#Compruebo en que difieren los años obtenidos del Case number con los registrados en YEAR, y se
# ve que tan solo hay 5 registros diferentes. Me quedo con la original (YEAR)--> DROP YEAR2

,Year,Year2
2449,1969,1989
3662,1971,1961
4312,1848,1948
4983,1922,1923
5043,1919,1900


In [25]:
sharks_copy.iloc[4312]
#Intento ver qué ha podido fallar.

Case Number                                            1948.09.17.R
Date                                           Reported 17-Sep-1848
Year                                                           1848
Type                                                     Unprovoked
Country                                                      TURKEY
Area                                                 Adana Province
Location                                                 Yumurtalik
Activity                                                   Swimming
Name                                                     Ali Kaymaz
Sex                                                               M
Age                                                             NaN
Injury                                                        FATAL
Fatal (Y/N)                                                       Y
Time                                                            NaN
Species                                         

In [26]:
sharks_copy.drop(['Year2'], axis=1, inplace = True)
sharks_copy.drop(['original order'], axis=1, inplace = True)
sharks_copy.drop(['Name'], axis=1, inplace = True)
sharks_copy.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal (Y/N),Time,Species,href formula,Month
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,16,Minor injury to thigh,N,13h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,9
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,36,Lacerations to hands,N,11h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,9
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,43,Lacerations to lower leg,N,10h43,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,9
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,9
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...,9


In [27]:
cols=sharks_copy.columns
print(cols)
cols = ['Case Number', 'Date', 'Year','Month', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'href formula']
sharks_copy = sharks_copy[cols]
sharks_copy.head()

#Reordeno columnas para que aparezca año y a continuación mes.

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ',
       'href formula', 'Month'],
      dtype='object')


,Case Number,Date,Year,Month,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal (Y/N),Time,Species,href formula
0,2016.09.18.c,18-Sep-16,2016,9,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,16,Minor injury to thigh,N,13h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2016.09.18.b,18-Sep-16,2016,9,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,36,Lacerations to hands,N,11h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
2,2016.09.18.a,18-Sep-16,2016,9,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,43,Lacerations to lower leg,N,10h43,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
3,2016.09.17,17-Sep-16,2016,9,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
4,2016.09.15,16-Sep-16,2016,9,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...


In [28]:
dict_meses= {
    1: 'Jan',
    2: 'Feb',
    3: 'Mar',
    4: 'Apr',
    5: 'May',
    6: 'Jun',
    7: 'Jul',
    8: 'Aug',
    9: 'Sep',
    10: 'Oct',
    11: 'Nov',
    12: 'Dic'
}

In [29]:
print(sharks_copy[(sharks_copy['Month']==0)&(sharks_copy['Year']>1950)][['Case Number','Year','Month', 'Date']].shape)
sharks_copy.shape

(158, 4)


(5868, 16)

In [30]:
#Tan solo hay 158 registros con Month == 0 posteriores a 1950--> DELETE todos los registros con month == 0
        #FINALMENTE NO LOS BORRO,PUESTO QUE HARÉ ANÁLISIS EN EL QUE NO NECESITO IMPERATIVAMENTE EL MES.
#cols_to_delete2= sharks_copy[(sharks_copy['Month']==0)]
#sharks_copy.drop(cols_to_delete2.index, axis=0, inplace = True)
#sharks_copy.shape

In [31]:
sharks_copy.describe()
#Vemos que el percentil 25 se sitúa en 1949, por lo que voy a desechar los datos anteriores a 1949

,Year,Month
count,5868.000000,5868.000000
mean,1965.887185,6.134799
std,61.730863,3.629566
min,5.000000,0.000000
25%,1945.000000,3.000000
50%,1976.000000,7.000000
75%,2004.000000,9.000000
max,2016.000000,12.000000


In [32]:
rows_to_delete3= sharks_copy[(sharks_copy['Year']<1949)]
sharks_copy.drop(rows_to_delete3.index, axis=0, inplace = True)
sharks_copy.shape

(4302, 16)

In [33]:
sharks_copy.head()

,Case Number,Date,Year,Month,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal (Y/N),Time,Species,href formula
0,2016.09.18.c,18-Sep-16,2016,9,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,16,Minor injury to thigh,N,13h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2016.09.18.b,18-Sep-16,2016,9,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,36,Lacerations to hands,N,11h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
2,2016.09.18.a,18-Sep-16,2016,9,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,43,Lacerations to lower leg,N,10h43,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
3,2016.09.17,17-Sep-16,2016,9,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
4,2016.09.15,16-Sep-16,2016,9,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...


In [38]:
display(sharks_copy[(sharks_copy['Month']==0)].shape) 
sharks_copy['Country'].value_counts().head()

(178, 16)

USA                 1782
AUSTRALIA            759
SOUTH AFRICA         455
PAPUA NEW GUINEA     114
BRAZIL                95
Name: Country, dtype: int64

In [39]:
list(set(sharks_copy['Fatal (Y/N)']))

[nan, 'N', ' N', 'UNKNOWN', 'Y']

In [ ]:
sharks_copy.columns

In [ ]:
sharks_copy.rename(str.strip, axis='columns', inplace=True)
sharks_copy.columns

In [ ]:
#sharks_0['Date'] = sharks_0['Date'].str.replace('?', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace(r'\w+efore ', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace('Beforer ', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace(r'Between', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace('Circa ', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace('Early ', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace(' or ', ',')
#sharks_0['Date'] = sharks_0['Date'].str.replace('-', ',')
#sharks_0['Date'] = sharks_0['Date'].str.replace('&', ',')
#sharks_0['Date'] = sharks_0['Date'].str.replace('Ca. ', '-')
#sharks_0['Date'] = sharks_0['Date'].str.replace(' B.C.', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace('.B.C..', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace('World War II', '1945')
#sharks_0['Date'] = sharks_0['Date'].str.replace('A few years ', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace('s', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace('No date, ', '')
#sharks_0['Date']

In [ ]:
#list(set(sharks_0['Date']))

In [ ]:
#sharks_0['Case Number']